# 客户端

这是一个示例客户端，与具有“认证(auth)”功能的服务器进行交互。

请参考服务器代码和FastAPI中的适当文档，以实现真正的安全性。

**注意** 仅服务器定义了调用端点！
因此，批量/流处理将不起作用。如果你想添加流和批量处理，也可以在服务器端进行相应的操作。
服务器使用APIHandler实现，它更加灵活，但确实需要更多的代码。 :）

## 作为 Alice 登录

In [1]:
import requests

response = requests.post("http://localhost:8000/token", data={"username": "alice", "password": "secret1"})
result = response.json()

In [2]:
token = result['access_token']

In [3]:
inputs = {"input": "hello"}
response = requests.post("http://localhost:8000/my_runnable/invoke", 
    json={
        'input': 'hello',
    },
    headers={
        'Authorization': f"Bearer {token}"
    }
)


In [4]:
response.json()

{'output': [{'page_content': 'cats like mice',
   'metadata': {'owner_id': 'alice'},
   'type': 'Document'},
  {'page_content': 'cats like cheese',
   'metadata': {'owner_id': 'alice'},
   'type': 'Document'}],
 'callback_events': [],
 'metadata': {'run_id': '1732c9aa-c6d3-4736-b8ca-01265fa8ba06'}}

您也可以通过 RemoteRunnable 接口与之交互（用于其他链中）

In [5]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/my_runnable", headers={"Authorization": f"Bearer {token}"})

In [6]:
await remote_runnable.ainvoke("cat")

[Document(page_content='cats like mice', metadata={'owner_id': 'alice'}),
 Document(page_content='cats like cheese', metadata={'owner_id': 'alice'})]

## 作为 John 登录

In [7]:
import requests

response = requests.post("http://localhost:8000/token", data={"username": "john", "password": "secret2"})
token = response.json()['access_token']
remote_runnable = RemoteRunnable("http://localhost:8000/my_runnable", headers={"Authorization": f"Bearer {token}"})

In [8]:
await remote_runnable.ainvoke("water")

[Document(page_content='i like walks by the ocean', metadata={'owner_id': 'john'}),
 Document(page_content='dogs like sticks', metadata={'owner_id': 'john'}),
 Document(page_content='my favorite food is cheese', metadata={'owner_id': 'john'})]